### Lybraries:

- Pymongo (MongoClient)

In [1]:
%pip install pymongo


Note: you may need to restart the kernel to use updated packages.


In [2]:
from pymongo import MongoClient
import pandas as pd

### Exploring collection: 'companies' 

Calling the collection from MongoCompass

In [3]:
client = MongoClient("localhost:27017")

In [4]:
db = client.get_database("ironhack")

In [5]:
c = db.get_collection("companies")

Searching for similar companies. Ideally: gaming, less than 100 employees, startups (less than five years since founding, collection was updated in 2013), web...
This way we will select the three that are more similar and explore their cities to match the conditions.

In [6]:
#We are going to stablish several conditions to find comparable companies, and realize querys to find the most similar ones
cond = {'founded_year':{'$gt':2008}}
cond1 = {'number_of_employees':{'$lte':100}}
cond6 = {'number_of_employees':{'$lte':100}}
cond2 ={'category_code': 'games_video'}
cond3 = {'category_code':'web'}
cond4 = {'category_code':'software'}
cond5 = {'category_code': 'mobile'}
cond6 = {'number_of_employees':{'$gte':30}}

#query = {'$and':[{'$or':[cond2,cond3,cond4,cond5]},cond, cond1,cond6]}
query = {'$and':[cond2,cond, cond1,cond6]}

proy = {'_id':0,'name':1, 'number_of_employees':1, 'founded_year':1, 'category_code' : 1, 'offices.city':1}

companies=list(c.find(query,proy).sort('offices.0.city',-1))

Lot of the companies found dont have the city information, so we are going to get the ones that have it

In [7]:
comparables = []
for i in companies:
    if len(i) > 4:
        comparables.append(i)

In [8]:
sim_comp = pd.DataFrame(comparables)


In [9]:
sim_comp

,name,category_code,number_of_employees,founded_year,offices
0,Tiny Speck,games_video,45,2009,[{'city': 'Vancouver'}]
1,Social Gaming Network,games_video,100,2011,"[{'city': 'Los Angeles'}, {'city': 'Beverly Hi..."
2,OneScreen,games_video,60,2009,[{'city': 'Irvine'}]


### Our three cities are
    - Vancouver for Tiny Speck
    - Beverly Hills for Social Gaming Network
    - Irvine  for OneScreen

In [13]:
sim_comp['latitude'] = [49.3023, 34.069722, 33.669445]

In [14]:
sim_comp['longitude'] = [-123.107, -118.405556, -117.823059]

In [19]:
sim_comp.to_csv('../data/similarcompanies.csv')